In [ ]:
# Question:
# Write a script for the below scenario either in PySpark (or) Spark Scala. 

# Note: 
# Code only using Spark RDD. 
# Dataframe or Dataset should not be used
# Candidate can use Spark of version 2.4 or above

# 1. Read the input testfile (Pipe delimited) provided as a "Spark RDD" 

# 2. Remove the Header Record from the RDD

# 3. Calculate Final_Price:

#        Final_Price = (Size * Price_SQ_FT)

# 4. Save the final rdd asTextfile with three fields

#         Property_ID|Location|Final_Price

In [1]:
pip install findspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init()

In [61]:
# Create SparkSession and SparkContext
from pyspark.sql import SparkSession
spark= SparkSession.builder\
                    .master("local")\
                    .appName('Assignment 2')\
                    .getOrCreate()

sc=spark.sparkContext

In [7]:
spark.version

'3.5.0'

In [62]:
# read the input file as RDD usinf spark context
input_file = "input_real_estate.txt"
rdd_in=sc.textFile(input_file)

In [63]:
# Retrieve and parse the header row
header = rdd_in.first()
header_names = header.split("|")

In [64]:
# Print the header names
print(header_names)

['Property_ID', 'Location', 'Price', 'Bedrooms', 'Bathrooms', 'Size', 'Price_SQ_FT', 'Status']


In [65]:
rdd_in.collect()

['Property_ID|Location|Price|Bedrooms|Bathrooms|Size|Price_SQ_FT|Status',
 '1461262|Arroyo Grande|795000|3|3|2371|365.3|Short Sale',
 '1478004|Paulo Pablo|399000|4|3|2818|163.59|Short Sale',
 '1486551|Paulo Pablo|545000|4|3|3032|179.75|Short Sale',
 '1492832|Santa Bay|909000|4|4|3540|286.78|Short Sale',
 '1499102|Thomas Country|109900|3|1|1249|98.99|Short Sale',
 '1489132|Thomas Country|109000|2|1|1129|93.99|Short Sale',
 '1467262|Fort Worth|987000|4|3|2771|465.3|Short Sale',
 '1478114|Paulo Pablo|409000|4|3|2918|223.19|Short Sale',
 '1402551|Nashville|545000|4|3|2932|169.75|Short Sale',
 '1405832|San Jose|980000|4|4|3340|290.98|Short Sale',
 '1493302|Fort Worth|119900|3|2|2249|198.99|Short Sale',
 '1412332|Thomas Country|129000|3|2|1329|73.99|Short Sale',
 '1469062|Arroyo Grande|798000|3|4|2321|235.9|Short Sale',
 '1498004|Nashville|789000|4|3|2419|263.59|Short Sale',
 '1586751|Nashville|896000|4|3|3132|199.75|Short Sale',
 '1433232|Glendale|987000|4|4|3340|216.78|Short Sale',
 '14955

In [78]:
# Apply filter to separate header and data
header = rdd_in.filter(lambda l: l.startswith("Property_ID"))
rdd1 = rdd_in.filter(lambda l: not l.startswith("Property_ID"))

In [79]:
# Apply Map and flatMap to get the column data
rdd2 = rdd1.flatMap(lambda x:x.split(',')).map(lambda x:x.split('|'))

In [80]:
rdd2.take(4)

[['1461262',
  'Arroyo Grande',
  '795000',
  '3',
  '3',
  '2371',
  '365.3',
  'Short Sale'],
 ['1478004',
  'Paulo Pablo',
  '399000',
  '4',
  '3',
  '2818',
  '163.59',
  'Short Sale'],
 ['1486551',
  'Paulo Pablo',
  '545000',
  '4',
  '3',
  '3032',
  '179.75',
  'Short Sale'],
 ['1492832', 'Santa Bay', '909000', '4', '4', '3540', '286.78', 'Short Sale']]

In [87]:
#Get the index of the required column
col_list=header.first().split('|')
f1=col_list.index("Property_ID")
f2=col_list.index("Location")
f3=col_list.index("Size")
f4=col_list.index("Price_SQ_FT")

In [86]:
#Function definition to calculate the final price
def mul_price(d1,d2):
    res=float(d1)*float(d2)
    return str(res)

In [90]:
#Call the function and create final result as expected
header_out=header.map(lambda x: x.split("|")[f1]+"|"+x.split("|")[f2]+"|Final_Price")
rdd3=rdd2.map(lambda x: x[f1]+"|"+x[f2]+"|"+ mul_price(x[f3],x[f4]))

final_out=header_out.union(rdd3)

In [100]:
final_out.collect()

['Property_ID|Location|Final_Price',
 '1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0',
 '1492832|Santa Bay|1015201.2',
 '1499102|Thomas Country|123638.51',
 '1489132|Thomas Country|106114.70999999999',
 '1467262|Fort Worth|1289346.3',
 '1478114|Paulo Pablo|651268.42',
 '1402551|Nashville|497707.0',
 '1405832|San Jose|971873.2000000001',
 '1493302|Fort Worth|447528.51',
 '1412332|Thomas Country|98332.70999999999',
 '1469062|Arroyo Grande|547523.9',
 '1498004|Nashville|637624.21',
 '1586751|Nashville|625617.0',
 '1433232|Glendale|724045.2',
 '1495502|Fort Worth|198377.13',
 '1489100|San Jose|62323.10999999999']

In [110]:
#Save the final Spark RDD as textfile
final_out.saveAsTextFile(r"C:\Users\shrid\Desktop\Apache Spark\solution.txt")

Py4JJavaError: An error occurred while calling o551.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/C:/Users/shrid/Desktop/Apache Spark/solution.txt already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:299)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:407)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.